# 成功的提示工程

---

在之前的章节中，我们已经构建了具备记忆功能的聊天机器人，并通过 RAG 将其连接到真实世界的数据，还利用**系统提示**将其调校为专业的 AI 助手。现在，是时候拓展我们对提示的认知了——提示并不仅仅是系统提示和用户查询。在一套 LLM 应用中，我们需要组合多种提示类型，引导模型输出符合预期的结果。

虽然**系统提示**定义了 LLM 的人格，但仍然存在许多关键问题等待解答：

* RAG 增强的 LLM 如何理解并使用从知识库检索到的上下文？
* 如何精调提示以获得最大的成效？

## 故事进展

用户非常喜欢 **TaskFriend**！不过，最近他们发现输出存在不一致的问题：**TaskFriend** 有时会跑题，或采用与预期不符的格式。简而言之——**TaskFriend** 缺乏明确的响应规范。

## 学习目标

* 理解提示模板如何引导 LLM 在 RAG 系统中使用检索到的上下文
* 学会运用 Role-Goal-Context-Audience-Format-Guardrails 框架设计高效的系统提示
* 掌握高级提示工程技巧：角色/人设提示、少样本学习、链式思维
* 使用元提示（Meta Prompting）自动改进提示
* 利用“LLM 作为裁判”技术，实现自动化评估与打分

## 初始化环境

### 配置 API Key

在任何笔记本开始工作前，我们都需要加载 [Model Studio 的 API Key](https://modelstudio.console.alibabacloud.com/?tab=globalset#/efm/api_key)，以便调用本课程中会持续使用的 Qwen 模型接口。

> 若不清楚如何获取 **Model Studio** API Key，请参考 `00 Setting Up the Environment` 文件。

In [ ]:
# Load Model Studio API key
import os
from config.load_key import load_key
load_key(
    confirmation=False
)

### 配置 LLM 与嵌入模型

我们将使用阿里云的 `qwen-plus` 作为 LLM，并搭配 DashScope 的 `text-embedding-v3` 作为嵌入模型。

本章节将把前面使用的 `OpenAI` 客户端替换为 `OpenAILike`。该客户端是 **LlamaIndex 专用封装**，适用于以下兼容 OpenAI 协议的模型：

* Model Studio
* DashScope
* vLLM
* Ollama
* 具备 OpenAI 兼容接口的本地 LLM

> **提示：** DashScope 的 API 端点为 `https://dashscope-intl.aliyuncs.com/api/v1`，不同于我们此前使用的 `https://dashscope-intl.aliyuncs.com/compatible-mode/v1`。

In [ ]:
# Set global settings
import time
import logging
import dashscope
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.dashscope import DashScopeEmbedding
from llama_index.llms.openai_like import OpenAILike
from pathlib import Path

logging.getLogger().setLevel(logging.ERROR)

# Dashscope uses https://dashscope-intl.aliyuncs.com/api/v1 
# instead of https://dashscope-intl.aliyuncs.com/compatible-mode/v1
dashscope.base_http_api_url ="https://dashscope-intl.aliyuncs.com/api/v1"

Settings.llm=OpenAILike(
    model="qwen-plus",
    api_base="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    is_chat_model=True
)

Settings.embed_model = DashScopeEmbedding(
    model_name="text-embedding-v3",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    encoding_format="float"
)

print("✅ Global parameters set!")

### 加载索引并启用 RAG

由于我们在前一章已经构建并保存了索引，此处只需从保存位置（`./knowledge_base/taskfriend`）加载即可。

我们将使用 `llama_index` 提供的以下组件：

* `StorageContext`：负责存储此前保存的节点、嵌入向量以及向量库，便于代码连接现有数据，从而无需重新处理文档即可恢复整个知识库。
* `load_index_from_storage`：基于 `StorageContext` 中的文件重建 RAG 索引，是快速恢复 RAG 系统的高效方法。

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

persist_path="./knowledge_base/taskfriend"

# Import index ("knowledgebase") we built last chapter,
storage_context = StorageContext.from_defaults(
    persist_dir=persist_path,
)

index = load_index_from_storage(
    storage_context,
    embed_model=Settings.embed_model
)
print(f"✅ Index loaded from `{persist_path}`!")

# Build the query engine (used to implement RAG)
query_engine = index.as_query_engine(
    streaming=True,
    llm=Settings.llm,
)
print("✅ Query engine built!")

我们已在上一章配置了 `get_qwen_stream_response` 函数，因此此处直接复用即可。

In [ ]:
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

def update_prompt_template(
        query_engine,
        # Use LlamaIndex prompt template as default template
        prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
    )):
    prompt_tmpl_str = prompt_tmpl_str
    qa_prompt_tmpl = PromptTemplate(prompt_tmpl_str)
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
    )
    return query_engine

def ask_llm(query, query_engine):
    streaming_response = query_engine.query(query)
    
    full_response = ""
    for token in streaming_response.response_gen:
        print(token, end="", flush=True)
        full_response += token

# 提示基础：如何构造优质提示

---

## 强力系统提示的组成（回顾）

在之前的章节中，我们已经了解到，**系统提示** 就是 AI 的**岗位说明书**。缺乏清晰指令时，模型往往会自由发挥，并产生不尽如人意的表现。

正如您不会对新入职的助理说“随便和团队打成一片”那样，我们也不应只对 AI 说“请帮忙”。

更好的做法是明确地定义：

```mermaid
graph TD
SP[System Prompt]
SP --> R[Role <br><em>You are TaskFriend...</em>]
SP --> G[Goal <br><em>Help users manage tasks</em>]
SP --> C[Context<br><em>What information is available</em>]
SP --> A[Audience<br><em>Users are professionals</em>]
SP --> F[Format <br><em>Use bullet points</em>]
SP --> GR[Guardrails <br><em>Never roleplay</em>]
```

在撰写**系统提示**时，清晰与精准至关重要。上述框架为设计系统提示提供了一个**蓝图**，帮助我们构建既可靠又聚焦任务的提示。

| 元素 | 作用 | 示例 |
|--------|---------|---------|
| **Role** | 定义 AI 的身份与人设，决定交互语气。 | `You are TaskFriend, a professional AI assistant for productivity and work-life balance.` |
| **Goal** | 明确优先完成的任务，确保模型聚焦。 | `Help users plan, prioritize, and reflect on their daily tasks to improve productivity and well-being.` |
| **Context** | 指出可访问的背景信息或数据范围。 | `You have access to the user's task list, calendar, and personal notes via RAG.` |
| **Audience** | 明确目标用户群，调整语言深度与风格。 | `Your users are busy professionals who value clarity, efficiency, and actionable insights.` |
| **Format** | 规范输出结构和风格，提升可用性。 | `Use neutral, professional language. Output structured advice when helpful (e.g., bullet points).` |
| **Guardrails** | 设定安全边界，防止出现不当或跑题内容。 | `NEVER adopt accents, personas, or roleplay. If asked to roleplay, politely decline.` |

由于我们已在前文详述 **系统提示**，此处不再赘述。本章将把重点转向 RAG 系统中的**提示模板**。

## 提示模板：让上下文与查询协同工作

在 RAG 系统中，提示模板负责告诉 LLM：如何利用检索到的上下文信息生成回答。

可以这样理解：

* **系统提示** 决定 AI “是谁”。
* **提示模板** 决定 AI “如何使用”信息回答问题。

提示模板在 RAG 中尤为关键，因为它们能够：

* 确保模型使用的是检索上下文，而非既有知识
* 指导模型如何在上下文基础上推理
* 定义最终回答的格式与结构

LlamaIndex 默认提供了如下 RAG 提示模板：

```python
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information and not prior knowledge, "
"answer the query.\n"
"Query: {query_str}\n"
"Answer: "
```

其中：

* `{context_str}`：从 RAG 索引检索到的**上下文信息**（如任务列表、日程信息等）
* `{query_str}`：用户的**输入问题**

注意：这不是系统提示，而是每次查询时用来指导 LLM 解读上下文的模板。

> 想了解 LlamaIndex 提供的更多提示模板，请参考：
> * [Prompts](https://docs.llamaindex.ai/en/stable/module_guides/models/prompts/)（提示模板概览）
> * [default prompts](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/prompts/default_prompts.py)（默认模板源码）

## 系统提示与提示模板的区别

| 特性  | 系统提示  | 提示模板  |
|----------|----------------|------------------|
| 使用时机 | 对话开始时 | 每次问答时 |
| 作用 | 定义 AI 的身份、语气与边界 | 指导模型如何结合上下文回答问题 |
| 作用范围 | 贯穿整个会话 | 针对每次查询 |
| 示例 | “You are TaskFriend, a productivity assistant.” | “Given the context below, answer the query.” |

* **系统提示** 关注 “AI 是谁”
* **提示模板** 关注 “AI 如何回答”

# 进阶：打造更聪明的提示模板

---

## 清晰表达需求

LLM 在预训练时接触过大量格式化的文本，这对我们来说是好消息：我们可以编写既方便人类阅读、又能让模型精准理解的提示。

以下是一些提升模板可读性的排版建议：

* 使用 `# 标题` 标注章节：
```markdown
# Important Instructions
```

* 使用分隔线（`---`）隔开上下文与查询：
```markdown
-------------------------
```

* 使用 \<标签> 或 [括号] 添加元数据：
```markdown
[Context Source: Task List]
<tag>Metadata</tag>
```

* 用 **粗体** 或 *斜体* 突出关键指令：
```markdown
**Do not use prior knowledge.** Only use the *context above*.
```

这些格式有助于人类与模型准确理解提示结构与意图。

接下来，我们为 **TaskFriend** 创建一个结构化的提示模板，以提升输出的清晰度与格式统一性：

In [ ]:
prompt_template = (
    "[Instructions] \n"
    "1. Do not make up tasks. \n"
    "2. You must emojis for each piece of information. \n"
    "3. Present information line-by-line. \n"
    "4. Only use information in the context to reply to user queries. \n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Query: {query_str}\n。"
    "Answer: "
)

update_prompt_template(query_engine,prompt_template)

In [ ]:
query = """
    Give me a random task from my task list.
"""
ask_llm(query,query_engine)

可以看到，**TaskFriend** 能严格遵循我们设定的格式要求。**提示模板** 的强大之处在于，它能在 RAG 系统中承担多种职责，灵活引导输出。

## 局限性

不过你会注意到，提示模板对于与上下文无关的指令并不十分“强势”。

试试看：

In [ ]:
query = """
    Give me a random task from my task list.
    Do not use emojis.
"""
ask_llm(query,query_engine)

In [ ]:
query = """
    Give me a task not from my task list.
    Do not use emojis.
"""
ask_llm(query,query_engine)

用户在查询中写下的任何指令，模型都会尽量遵从——哪怕会覆盖我们在模板中设定的部分规则。

不过，只要涉及上下文内容，你会发现提示模板仍能发挥作用。

## 为什么仍要使用提示模板？

### 它们是开发者工具，而非防火墙

提示模板的目标并非阻止用户或强制执行行为，而是为了：

* 标准化传递给 LLM 的输入
* 提升输出的一致性
* 让提示工程更易复用与测试

### 它们能提升 RAG 回答质量

在 RAG 系统中，提示模板是教会模型“如何使用上下文”的关键。

若没有优秀的模板，模型可能会：

* 忽视上下文
* 退回到自身训练知识
* 给出含糊或泛泛的答案

而设计良好的模板能够确保模型确实利用了检索到的知识。

### 它们能强化结构化输出

通过提示模板，我们可以引导模型输出：

* JSON
* 要点列表
* 表格
* 代码
* Markdown

这样有助于下游应用解析与调度模型输出。更棒的是，这类结构化约束在提示模板中非常可靠：

In [ ]:
prompt_template = (
    "[Instructions] \n"
    "1. Do not make up tasks. \n"
    "2. Only use information in the context to reply to user queries. \n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "[Output format] \n"
    "Answer in JSON format with the following structure:\n"
    "{{\n"
    "  'task': string,\n"
    "  'due_date': string,\n"
    "  'priority': string\n"
    "}}\n"
    "Query: {query_str}\n"
    "Answer: "
)

update_prompt_template(query_engine,prompt_template)

In [ ]:
query = """
    Give me a task from my task list.
    Format each item as a bullet point.
"""
ask_llm(query,query_engine)

# 通用提示工程技巧：优化 AI 行为

---

前文我们已经讨论了 **系统提示** 与 **提示模板**。接下来，将把焦点放在“提示本身能做到什么”以及“如何迭代提示以持续改进效果”。掌握这些技巧后，我们就可以将其整合进用户不可见的系统提示与模板之中。

> **提示：** 为了演示方便，本节将以“用户查询”的方式编写提示，而不是直接修改系统提示或模板，从而减少所需的代码改动。

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
)

def get_qwen_stream_response(query):
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=[{"role": "user", "content": query}],
        stream=True
    )
    full_response = ""
    
    for chunk in completion:
        try:
            content = chunk.choices[0].delta.content
            if content:
                print(content, end="", flush=True)
                full_response += content
        except Exception as e:
            pass
        time.sleep(0)
    
    print()
    return full_response

## 角色与人设

您的 LLM 应用理应具备独特的风格，它的说话方式与处理结果都应该符合特定场景需求。

在产品设计中，这一点尤为重要：应用定位会直接影响用户感知。

我们用一个简单实验来感受差异：

In [ ]:
# Set the question
question = """
    What is Qwen?
"""

In [ ]:
# Let LLM take on the role of a professional LLM developer
pseudo_template = """
    You are a professional LLM developer.
    Your job is to explain LLM technology and concepts in-depth.
    Limit your explaination to 100 words.
"""

query = pseudo_template + question

response = get_qwen_stream_response(query)

In [ ]:
# Let LLM take on the role of a kindergarden teacher
pseudo_template = """
    You are a kindergarden teacher.
    Your job is to explain complex ideas in easy-to-understand comparisons.
    Explain like your audience is 5 years old.
    Limit your explaination to 100 words.
"""

query = pseudo_template + question

response = get_qwen_stream_response(query)

预期输出：

| 角色/人设  | LLM 开发者  | 幼儿园老师  |
|-------|----------------|------------------------|
| 提示 | You are a professional LLM developer.<br>Your job is to explain LLM technology and concepts in-depth.<br>Limit your explaination to 100 words.  | You are a kindergarden teacher.<br>Your job is to explain complex ideas in easy-to-understand comparisons.<br>Explain like your audience is 5 years old.<br>Limit your explaination to 100 words.  |
| 查询 | What is Qwen?  | What is Qwen?  |
| 输出  | Qwen is a large language model (LLM) developed by Alibaba Cloud, designed to understand and generate human-like text across multiple languages and domains. It leverages deep learning techniques, particularly transformer architectures, to process input and produce coherent, contextually relevant responses. Qwen supports various tasks such as translation, question-answering, content creation, and code generation, making it versatile for both research and practical applications.  | Qwen is like a super-smart robot friend who lives inside a computer. Just like how you ask your teacher questions, people can ask Qwen all sorts of questions, and it tries to give helpful answers! It's like having a magic helper that knows lots of things and can talk to you through typing. But remember, it's not a real person - it's just really good at pretending to be one when it helps you learn or solve problems!  |

可以看到，只需简单的提示工程，就能让 LLM 以指定角色或人设的方式“说话”。这为我们设定应用调性奠定了基础。

## 单样本与少样本学习

我们还可以把 One-shot / Few-shot 示例直接写入提示中：

In [ ]:
# Set the question
question = """
    Recipe for Old Fashioned.
"""

In [ ]:
# Let the LLM work out on its own
pseudo_template = """
    [Goal]
    Create content for user based on input
    ---------
    [Output format]
    1. Markdown format
    2. Use properly formatted lists and tables
    ---------
    [User input]
    The following requirement is provided by the user:
"""

query = pseudo_template + question

response = get_qwen_stream_response(query)

In [ ]:
# Provide one-and-few-shot example
pseudo_template = """
    [Goal]
    Create content for user based on input
    ---------
    [Output format]
    1. Markdown format
    2. Use properly formatted lists and tables
    ---------
    [Example]
    # Recipe name
    Short description of recipe
    
    ## Ingredients
    1. x tbsp - Ingredient 1
    2. y tsp - Ingredient 2
    
    ## Instructions
    1. Do X
    2. Do Y
    
    ## Tips
    ---
    [User input]
    The following requirement is provided by the user:
"""

query = pseudo_template + question

response = get_qwen_stream_response(query)

这很直观——示例会引导模型控制内容的范围与输出格式。

## 链式思维（Chain-of-Thought）

我们同样可以通过提示让 LLM 显式（或隐式）执行链式思维：

In [ ]:
question = """
    A large jar contains a mix of coins: only pennies (1¢), nickels (5¢), and dimes (10¢).
    
    There are:
    * 100 coins in total,
    * The total value of the coins is exactly $5.00 (that is, 500 cents),
    * The number of pennies is equal to the number of nickels and dimes combined.
    
    How many of each type of coin are in the jar?
"""

In [ ]:
pseudo_template = """
    Help the user solve math problems
    Give the answer directly, DO NOT show your work
"""

query = pseudo_template + question

response = get_qwen_stream_response(query)

In [ ]:
pseudo_template = """
    Help the user solve math problems
    Show your work step-by-step, grouped by logical sections
"""

query = pseudo_template + question

response = get_qwen_stream_response(query)

这是个颇有意思的实验：

* 不展示推理过程时，模型可能给出**错误**答案
* 展示推理过程时，模型往往能得出**正确**答案

Chain-of-Thought（又称 **Reasoning**）能够帮助模型“自我思考”，提升答案质量。

## 彩蛋：推理模型

延续上一节内容，我们来看看具备推理能力的模型。诸如 Qwen3（思考模式）、基于 Qwen2.5 的 QwQ、DeepSeek-R1 等模型，都可以在给出最终答案前输出完整的推理过程。

> 想了解 Model Studio 上的推理模型，请参阅 [Deep Thinking](https://www.alibabacloud.com/help/en/model-studio/deep-thinking)。

In [ ]:
# Create reasoning function
def get_qwen_reasoning(query, verbose=True):
    
    
    completion = client.chat.completions.create(
        model="qwen-plus-2025-04-28",                       # ← Use the "qwen-plus-2025-04-28" model
        messages = [{"role": "user", "content": query}],
        extra_body={
            "enable_thinking": True,                        # ← Enable thinking
            # "thinking_budget": 200                          # ← Enable to limit tokens used for thinking
        },
        stream=True,
    )

    reasoning_content = ""
    answer_content = ""
    is_answering = False
    print("\n" + "=" * 20 + "Thinking Process" + "=" * 20 + "\n")

    for chunk in completion:
        if not chunk.choices:
            continue

        delta = chunk.choices[0].delta

        # Capture reasoning content
        if hasattr(delta, "reasoning_content") and delta.reasoning_content is not None:
            if verbose:
                print(delta.reasoning_content, end="", flush=True)
            reasoning_content += delta.reasoning_content

        # Capture final answer
        if hasattr(delta, "content") and delta.content:
            if not is_answering and verbose:
                print("\n" + "=" * 20 + "Complete Response" + "=" * 20 + "\n")
                is_answering = True
            if verbose:
                print(delta.content, end="", flush=True)
            answer_content += delta.content

In [ ]:
query = """
    Why is the sky blue?
"""

get_qwen_reasoning(query)

### 使用推理模型的小贴士

虽然推理模型功能强大，但它们在推理过程中也可能给用户带来困扰。本节提供一些实用建议，帮助你更好地驾驭这类模型。

#### 使用清晰、准确的提示

面对推理模型时，保持提示的清晰度与准确性尤为重要，这能避免模型“神游”，并促使它输出对解决任务真正有帮助的思考过程：

In [ ]:
# A piece of code with no clear requirements on what the user expects
query = """
def example(a):
  b = []
  for i in range(len(a)):
    b.append(a[i]*2)
  return sum(b)
"""

get_qwen_reasoning(query)

In [ ]:
# The same piece of code with requirements clearly stated
query = """
What does this python code do, and how can I simplify it? Be concise.
def example(a):
  b = []
  for i in range(len(a)):
    b.append(a[i]*2)
  return sum(b)
"""

get_qwen_reasoning(query)

#### 谨慎使用链式思维提示

由于推理模型在“思考”阶段本身就存在链式思维成分，再次要求它执行链式推理，可能导致循环或冗余。随着 LLM 不断升级，这类问题有望逐步被优化。

# 元提示：让 LLM 自动写出更好的提示

---

元提示（Meta Prompting）是一种提示技巧：你向 LLM 提问，但问题的核心是“如何回答提示”。换言之，就是“关于提示的提示”——“meta” 即“超越”或“关于自身”的意思。

与其直接提问，不如先指导模型如何思考、如何组织结构，再给出实际任务。这样可以显著提升回答的质量、准确性与一致性。

## 为什么它有用

编写高质量提示是一项需要迭代的技能——即使是专家也难以一次到位。通常流程如下：

```mermaid
graph LR
    A[Write Initial Prompt] --> B[Run LLM]
    B --> C[Analyze Output]
    C --> D{Meets Quality?}
    D -- No --> E[Generate Feedback via LLM Critic]
    E --> F[Suggest Improvements]
    F --> G[Rewrite Prompt]
    G --> B
    D -- Yes --> H[Deploy Optimized Prompt]
```

元提示的优势在于：

* **反馈系统化：** 利用 LLM 作为稳定的“裁判”（LLM-as-a-judge）。
* **迭代更迅速：** 从想法到优化只需数秒。
* **提示进入 CI/CD：** 像管理代码一样管理提示——测试、评分、版本化、部署。

这让提示工程摆脱“猜测式调参”，转而成为**可重复、可扩展的工作流程**，对生产级 AI 系统至关重要。

为了深入理解元提示的威力，我们马上动手实践。

> **提示：** 为确保示例一致，下面我们将模拟固定的检索上下文。

## 第一步：你的“初始”提示

所有优化都有起点。所谓“初始提示”通常结构松散、缺乏语气和格式要求。

它可以“很糟糕”——这正是我们需要改进它的原因。

In [ ]:
context  = """
    TaskFriend is a productivity assistant that helps users manage tasks, calendars, and notes.
    It supports task prioritization, deadline tracking, and integration with external calendars like Google Calendar and Outlook.
    Users can add tasks via voice, text, or email. TaskFriend automatically sorts tasks by due date and priority.
    It also provides daily summaries and weekly planning tools to help users stay on top of their workload.
"""

first_query = f"""
    Based on the following information, describe in what TaskFriend is and what it does in a paragraph.

    [Reference Info]
    {context}
"""

print("-" * 50)
print("'First' (naive) answer")
print("-" * 50 + "\n")

first_response = get_qwen_stream_response(first_query)

**你可能得到的结果：**  
结构平淡、缺乏层次——往往是一整段没有视觉元素的文字，与上下文几乎没有差异，例如：

> `TaskFriend is a comprehensive productivity assistant designed to help users efficiently manage their tasks, calendars, and notes. It enables users to add tasks through voice, text, or email, and automatically organizes them by due date and priority, ensuring important deadlines are never missed. With integrations for popular external calendars like Google Calendar and Outlook, TaskFriend streamlines scheduling and time management. Additionally, it offers daily summaries and weekly planning tools to keep users informed and prepared, making it easier to stay on top of their workload and maintain productivity.`

**为什么重要：**  
这是基线。许多人会止步于此。但在生产环境中，这恰恰是“真正工程工作”的起点。

> **💡 核心洞见：**  
> 粗糙的提示只会产生粗糙的输出。若想优化结果，必须先界定“更好”意味着什么。

在以下示例中，我们将从一个简单的提示开始，并运用这四个步骤逐步改进它。这是一个常见但结构松散的 TaskFriend 功能提示：

In [ ]:
meta_prompt = f"""
    [Role] You are a prompt engineering expert. 
    [Task] Your job is to help me write a prompt based on the instructions below.

    [Original prompt]
    ---
    {first_query}
    ---

    [Generated response]
    ---
    {first_response}
    ---

    [Instructions]
    The response is too basic. I want the assistant to sound professional yet friendly, and to clearly explain TaskFriend’s value to a new user.

    Specific improvements the prompt needs to achieve:
    1. **Tone**: Friendly, encouraging, and helpful — like a real productivity buddy.
    2. **Structure**: Use bullet points or categories (e.g., "What It Is", "Key Features", "Why It Helps").
    3. **Visual Appeal**: Use emojis to highlight features and draw attention.

    Output only the prompt, do not output any explainations or context.
"""

print("-" * 50)
print("Optimized prompt")
print("-" * 50 + "\n")

optimized_prompt = get_qwen_stream_response(meta_prompt)
print(optimized_prompt)

让我们仅运行 `run_assistant()` 来查看结果：

{% raw %}
```

(((<some generic output>)))
```

{% endraw %}

结果比较一般，可以看到：

- 输出不完整——只好像是某个长段落的开头。
- 表述非常笼统。
- 全程没有结构或项目符号。

我们把这当成起点。很多时候，你设计提示时会觉得“还行”——但系统提示未做任何改进。

## 步骤 2：设计提示策略

In [ ]:
final_prompt = f"""
    Based on the following information, describe TaskFriend in a way that’s professional yet friendly — like a helpful productivity buddy. 
    Organize your response into clear sections such as **"What It Is"**, **"Key Features"**, and **"Why It Helps"**.
    Use bullet points or emojis (like 📅, ⏰, 🎯) to make it visually engaging and highlight important features. 
    Make sure new users clearly understand what TaskFriend does and why it's valuable for staying organized and productive.

    [Reference Info]
    {context}
"""
print("-" * 50)
print("Final answer (with optimized prompt)")
print("-" * 50 + "\n")

final_answer = get_qwen_stream_response(final_prompt)
print(final_answer)

就像我们在上一章构建多轮对话时所做的那样，我们可以创建提示模板来改进结构、变量以及我们传递给模型的上下文。

我们从一个更强的提示模板开始：

这里我们做了几件事：

- **以清晰的结构开始，强调我们想要的内容**。在这个示例中，我们明确传达：我们希望得到一个营销简介，它分为“概述”和“关键功能”两个部分。
- 我们**提供多个要素的定义**：我们包括产品或功能的名称、用户角色、分类，以及我们需要涵盖的关键功能。这些要素中的很多来自我们记录的输入。
- 我们**留有一个“用户请求”槽位**：这样我们可以将用户的输入传递进去。
- 我们**声明期望的输出格式**：第一项要求为两到三段，并附带项目符号列表。

In [ ]:
import json
from functions.clean_json import clean_json_response

reference_answer = """
👋 Meet TaskFriend — Your Personal Productivity Partner!

📌 **What It Is**  
TaskFriend is a smart productivity assistant that helps you manage tasks, calendars, and personal notes — all in one place.

⚙️ **Key Features**  
- 📅 Syncs with Google Calendar and Outlook  
- 📝 Takes notes and turns them into actionable tasks  
- 🧠 Prioritizes your to-dos based on urgency and importance  
- 🗣️ Supports voice, text, and email input  
- 📋 Daily summaries and weekly planning tools  

✨ **Why It Helps**  
TaskFriend keeps your workflow smooth and your mind clear — so you can focus on what matters most.
"""

def get_qwen_response(query):
    full_response = ""
    for token in get_qwen_stream_response(query):
        full_response += token
    return full_response

def analyze_gap_quick(generated_response, reference):
    gap_prompt = f"""
    [Role] You are a prompt evaluation assistant.
    [Task] Compare the generated response to the reference answer. Be sensitive to improvements.

    [Instructions]
    - Compare ONLY the generated response and reference below.
    - Score each dimension 1–5 (5 = perfect match to reference in style and substance).
    - Focus on visual layout, emoji use, section headers, conciseness, and scannability.
    - If the current response better matches the reference than before, reflect that in higher scores.

    [Scoring Rubric]
    - tone: Friendly, upbeat, conversational (like a helpful friend). Emojis enhance warmth.
    - structure: Sections titled exactly or similarly: "What It Is", "Key Features", "Why It Helps"
    - content: All key features and benefits present without extra fluff
    - visual: Uses emojis, bolding, bullet points, spacing like the reference — clean and easy to scan

    [Critical] Re-evaluate completely. Do not repeat previous scores. Even small improvements matter.

    [Reference Answer]
    {reference}

    [Generated Response]
    {generated_response}

    [Output Format]
    Return a JSON object like:
    {{
      "tone": int,
      "structure": int,
      "content": int,
      "visual": int
    }}
    """
    return get_qwen_response(gap_prompt)

def suggest_prompt_improvements(current_prompt, gap_scores, reference):
    improvement_prompt = f"""
    [Role] You are a prompt engineering expert.
    [Task] Based on the gap scores, suggest specific changes to elements that score <4 to improve alignment with the reference answer.

    [Current Prompt]
    {current_prompt}

    [Gap Scores]
    {json.dumps(gap_scores)}
    
    [Reference Answer]
    {reference}

    [Instructions]
    Return a list of actionable suggestions, be short and concise, 5 actions max. (e.g., "Add 'use emojis' to instructions", "Clarify feature list").
    """
    return get_qwen_response(improvement_prompt)

def improve_prompt_with_suggestions(current_prompt, suggestions):
    improvement_application_prompt = f"""
    [Role] You are a prompt engineer.
    [Task] Improve the following prompt based on the given suggestions.

    [Current Prompt]
    {current_prompt}

    [Suggestions]
    {suggestions}

    [Instructions]
    Return the improved prompt only — no explanation.
    """
    return get_qwen_response(improvement_application_prompt)

current_prompt = first_query  # Make sure this is defined earlier
prompt_history = []  # Optional: track prompt evolution

for i in range(3):  # Max 3 iterations
    print("\n" + "=" * 50)
    print(f"Iteration {i+1}")
    print("=" * 50)

    # Get and print the full streamed response
    generated_response = get_qwen_stream_response(current_prompt)
    print(current_prompt)
    print(f"\n\nGenerated Response (sample):\n{generated_response[:250]}...")

    # Analyze gap with lightweight scoring
    gap_report = analyze_gap_quick(generated_response, reference_answer)

    # Clean the response before parsing
    cleaned_report = clean_json_response(gap_report)

    try:
        gap_scores = json.loads(cleaned_report)
        print("\n✅ Parsed gap scores:", gap_scores)

        # Check for convergence
        if all(score >= 4 for score in gap_scores.values()):
            print("\n✅ Evaluation passed. Output quality is sufficient.")
            break

        # Get improvement suggestions
        print("\n🔍 Generating prompt improvement suggestions...")
        print("Suggested Improvements:\n")
        improvement_suggestions = suggest_prompt_improvements(current_prompt, gap_scores, reference_answer)
        print(improvement_suggestions)
        print("\n")
              
        # Apply suggestions to current prompt
        print("\n🛠️ Applying suggestions to improve the prompt...\n")
        print("Improved prompt:")
        current_prompt = improve_prompt_with_suggestions(current_prompt, improvement_suggestions)
        print(current_prompt)
        print("\n")
        
        # Optional: Store in history
        prompt_history.append({
            "iteration": i+1,
            "prompt": current_prompt,
            "gap_scores": gap_scores
        })

    except json.JSONDecodeError:
        print("[Error] Could not parse gap scores. Skipping convergence check.")
        print("Raw gap report:", gap_report)
        continue
else:
    print("\n🔁 Max iterations reached. Optimization complete.")

这已经比之前好很多了！庆祝一下——在提示工程中保持动力很重要。

In [ ]:
from functions.grader_plot import plot_grader
import re


# Test Responses
bad_response = """
**TaskFriend** is a productivity assistant designed to help users efficiently manage their time.
It offers a range of features aimed at improving organization and time management:

- **Task Management**
- **Deadline Tracking**
- **Calendar Integration**
- **Planning & Summaries**

Overall, TaskFriend serves as a comprehensive organizational tool that helps users streamline 
their workflow, reduce missed deadlines, and maintain better control over their daily and 
long-term tasks.
"""


ok_response = """
👋 I'm **TaskFriend**, your personal productivity partner — nice to meet you! 🎉

I’m here to help you take control of your tasks, calendars, and notes — all in one smart, 
easy-to-use place. Let’s make sure you’re always working on what matters most, without the stress.  

Here’s how I can help boost your productivity:

### 📅 Smart Scheduling  
Sync seamlessly with **Google Calendar** and **Outlook**, so your schedule is always up-to-date and at your fingertips.

### 📝 Flexible Task Entry  
Add tasks using:
- ✍️ Text
- 🗣️ Voice input
- 📧 Email integration  
Whatever fits your style best!

### 🔍 Automatic Prioritization  
Let me sort your to-dos by **due date** and **importance**, so you know exactly where to focus each day.

### 🧠 Daily & Weekly Planning Tools  
Start your day strong with a clear plan and get ahead on the week — I’ll help you stay strategic and organized.

"""

# Re-use previous reference answer, exact match
perfect_response = reference_answer

# Grade Response Quality
def grade_response_detailed(response_to_grade, reference=reference_answer):
    grader_prompt = f"""
    [Role] You are a prompt evaluation assistant.
    [Task] Compare the generated response to the reference answer. Be sensitive to improvements.
    
    [Instructions] Maximum score for each metric is 5, while the minimum is 1

    [Reference answer]
    {reference}

    [Generated response]
    {response_to_grade}

    [Output Format]
    Return only a JSON object like:
    {{
      "tone": int,
      "structure": int,
      "content": int,
      "visual": int
    }}
    
    Do not return anything else
    """
    try:
        raw_output = get_qwen_response(grader_prompt)

        # Extract JSON silently
        json_match = re.search(r'\{[^}]+\}', raw_output, re.DOTALL)
        if json_match:
            cleaned = json_match.group(0)
            result = json.loads(cleaned)
            # Clamp scores to 1–5 (fallback measure)
            return {k: max(1, min(5, v)) for k, v in result.items()}
        else:
            return {"tone": 1, "structure": 1, "content": 1, "visual": 1}
    except Exception:
        return {"tone": 1, "structure": 1, "content": 1, "visual": 1}

# 📊 Evaluate Responses
scores = {
    "Initial Prompt": grade_response_detailed(bad_response),
    "Optimized Prompt": grade_response_detailed(ok_response),
    "Final Prompt": grade_response_detailed(perfect_response),
}

# 📈 Plot Results
plot_grader(
    scores,
    title="Prompt Evolution: Response Quality Over Iterations"
)

<b>1. 在 RAG 系统中，提示模板的主要目的是什么？</b>  

<ul>
    <li>A) 存储检索到的文档    </li>
    <li>B) 定义 AI 的人格特质    </li>
    <li>C) 指示 LLM 如何利用检索到的上下文回答查询    </li>
    <li>D) 取代系统提示</li>
</ul>

**查看答案 →**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** C) 指示 LLM 如何利用检索到的上下文回答查询  
📝 **说明**：
* 提示模板通过组合检索到的上下文和用户查询来组织传入 LLM 的输入，确保模型基于提供的数据作答，而不是依赖其既有知识。

</div>
</details>



## 要点回顾

* **什么是优质提示**
    * **系统提示就是 AI 的岗位说明书**——它界定角色、目标、受众、输出格式以及安全护栏。
    * **强提示可以防止偏航**——没有明确指令时，LLM 容易给出泛泛或跑题的回应。
    * **使用 R-G-C-A-F-G 框架**：
      - **Role（角色）**：AI 是谁（例如：“TaskFriend，一名效率助理”）
      - **Goal（目标）**：它要达成什么（例如：“帮助用户为任务排优先级”）
      - **Context（上下文）**：它可以使用哪些数据（例如：“通过任务列表的 RAG 检索”）
      - **Audience（受众）**：它在与谁对话（例如：“忙碌的职场人士”）
      - **Format（格式）**：它应该如何回应（例如：“使用项目符号，不要 Markdown”）
      - **Guardrails（护栏）**：它必须避免什么（例如：“禁止角色扮演，不发表个人意见”）
    * **提示模板的作用不同**——它们约束的是 LLM 如何利用**检索到的上下文**，而非它的身份。

<br>

* **用提示模板连接上下文与查询**
    * **提示模板是 RAG 的引擎**——它指示 LLM 如何利用检索文档作答。
    * **它们强化基于依据的输出**——类似“请基于上下文信息作答，不要引用既有知识”的语句能减少幻觉。
    * **它们标准化输入结构**——让提示更可预测、更易测试。
    * **优秀的模板包含以下要素**：
      - 在上下文与查询之间做清晰分隔（例如 `---`）
      - 指示忽略既有知识
      - 输出格式规则（例如 JSON、项目符号）
    * **它们并非万无一失**——用户查询仍可能覆盖模板指令，因此要明确设计边界。

<br>

* **构建更智能的提示模板**
    * **格式化提升清晰度**——使用 `# 标题`、`---`、`[方括号]` 和 `**加粗**` 引导模型。
    * **表情和视觉提示改善体验**——让输出更易读、更有吸引力。
    * **结构化输出至关重要**——提示模板可强制生成 JSON、表格或代码，便于生产系统解析。
    * **提示模板是开发者工具**——它们提升一致性，但无法完全阻止用户输入的影响。
    * **它们让 RAG 更可靠**——没有模板时，模型可能忽视上下文或退回到训练数据。

<br>

* **通用提示工程技巧**

    * **角色/人设提示**可改变语气与深度——例如“用 5 岁儿童听得懂的方式解释”与“以资深工程师的角度说明”。
    * **少样本示范**用示例教授格式和风格——哪怕只有一个示例，也能显著提升输出一致性。
    * **链式思考（CoT）**能提升推理能力——让模型“展示推理过程”往往能得到更好的答案。
    * **推理模型（如启用思考模式的 Qwen-plus）**更进一步——它们会在回答前自动生成内部推理。
    * **在推理模型上避免强制 CoT 提示**——可能造成循环思考；模型已在内部进行推理。

<br>

* **元提示与自动化评估**
    * **元提示**即“围绕提示本身进行提示”——利用 LLM 来审查并改进你的提示。
    * **它能加速迭代**——无需反复人工试错，而是自动化优化循环。
    * **差距分析**通过 LLM 评审将输出与参考答案对比。
    * **自动评分**使用评分量表（语气、结构、内容、视觉）量化质量。
    * **让提示也能 CI/CD**——像代码一样测试、打分和发布提示更新，确保大规模的一致质量。